In [15]:
#import libraries
import os
from os import listdir
from os.path import isfile, join
from docx2python import docx2python
import pymupdf,fitz
import platform
import spacy

In [16]:
#output json array


In [49]:
#nlp Processing
nlp = spacy.load("en_core_web_lg")
#add pipeline
ruler=nlp.add_pipe("entity_ruler",before="ner")
#ruler pipeline add pattern
skills = "skill_pattern.jsonl"
ruler.from_disk(skills)
def processTextUsingSpacy(inputText, fileName):
    mobileNumber = ''
    #re find(mobile-pattern)
    #check list empty
    #if not store mobile in variable
    doc = nlp(inputText)
    skills=set()
    required_skill={'.net', 'java','llm'}
    detected_req_skill_count=0
    name='empty'
    email=''
    mobile = ''
    
    for ent in doc.ents:
        #print('ent.label:', ent.label_, 'text:', ent.text)
        if(ent.label_ == 'SKILL'):
            skills.add(ent.text.lower())
        if (ent.label_ == 'EMAIL'):
            email = ent.text.lower()
        if(ent.label_ == 'MOBILE'):
            mobile = ent.text
        if(ent.label_ == 'CARDINAL') and len(ent.text) == 10:
            mobile = ent.text
        if (ent.label_ == 'DATE') and len(ent.text) == 10:
            mobile = ent.text
            
    for ent in doc.ents:
        if (ent.label_ == 'PERSON'):
            if(len(ent.text) > 4):
                shortName = ent.text[:3].lower()
                if(shortName in email):
                    name = ent.text
    print("\n")
    print('\033[1m'+'\033[94m'+"------Resume:",fileName,"NLP Parser Output-------"+'\033[0m')
    if(len(skills) > 0):
        print(name,'candidate skills:',skills)
    else:
        print(name," has none of required skills")
    print('required skills:',required_skill)
    print('email:',email)
    if(mobile == ''):
        print("Mobile number not detected.")
    else:
        print('mobile:', mobile)
    candidateSelected = True
    for x in required_skill:
        if(not x in skills):
            candidateSelected = False           
    if candidateSelected:
        print('\033[1m'+'\033[92m'+name,' selected'+'\033[0m')
    else:
        print('\033[1m'+'\033[91m'+name,' rejected'+'\033[0m')
    #create a json string
    #insert json string into array

In [50]:
#clear jsonArray
cwd = os.getcwd()
path = os.path.join(cwd, "resume")
fileList = [f for f in os.listdir(path)]
print('Total Number of resumes to be processed:',len(fileList))
for file in fileList:
    if('.docx' in file):
        # extract docx content
        wordfile = os.path.join(cwd, "resume",file)
        with docx2python(wordfile) as docx_content:
            processTextUsingSpacy(docx_content.text, file)
    if('.pdf' in file):
        pdfFile = os.path.join(cwd, "resume",file)
        doc=fitz.open(pdfFile)
        text=""
        for page in doc:
            text = text+str(page.get_text())
        tx ="".join(text.split("\n"))
        doc.close()       
        processTextUsingSpacy(tx,file)


Total Number of resumes to be processed: 2


------Resume: aswin_resume.docx NLP Parser Output-------
Aswin Loganathan candidate skills: {'.net', 'llm', 'java'}
required skills: {'.net', 'llm', 'java'}
email: aswin.l@company.net
mobile: 123 456 7890
Aswin Loganathan  selected


------Resume: Data Scientist.pdf NLP Parser Output-------
Jane Smith  has none of required skills
required skills: {'.net', 'llm', 'java'}
email: janesmith@example.com
Mobile number not detected.
Jane Smith  rejected


In [ ]:
#Print user readable Results by parsing array of jsonString